In [1]:
import google.generativeai as genai
import ast
import os
import json
import PIL.Image
from selenium.webdriver.common.by import By
from constants import *

# Load environment variables
from dotenv import load_dotenv
from selenium_functions import open_browser

load_dotenv()

# Convert the GEMINI_API_KEYS string from environment variables to a list
GEMINI_API_KEYS = os.environ.get("GEMINI_API_KEYS")
KEY_LIST = ast.literal_eval(GEMINI_API_KEYS)

# Randomly shuffle the list of API keys
import random
random.shuffle(KEY_LIST)

# Global index to keep track of the current key
current_api_key_index = 0

c:\Users\Dylan\VSCode\projects\GoogleXMHacks\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from lxml import html


def extract_elements_by_xpath(html_string, xpath_selector):
    # Parse the HTML
    tree = html.fromstring(html_string)

    # Apply the XPath selector
    elements = tree.xpath(xpath_selector)

    # Return a list of outer HTML for each element
    return str([html.tostring(element).decode("utf-8") for element in elements] + [xpath_selector])

In [4]:
def cycle_api_key():
    global current_api_key_index
    if current_api_key_index >= len(KEY_LIST) - 1:
        current_api_key_index = 0
    else:
        current_api_key_index += 1
    return KEY_LIST[current_api_key_index]


def generate_content_with_cycling_keys(prompt, system_prompt, image=None):
    global current_api_key_index
    # Get the current API key and cycle to the next one for future requests
    api_key = cycle_api_key()

    # Configure the generative AI model with the new API key
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel(
        "gemini-1.5-pro-latest",
        generation_config=genai.GenerationConfig(
            max_output_tokens=8000,
            temperature=0,
        ),
        system_instruction=system_prompt,
    )

    # Generate content using the provided prompt
    if image is None:
        response = model.generate_content(prompt, request_options={"timeout": 1000})
    else:
        response = model.generate_content(
            [prompt, image], request_options={"timeout": 1000}
        )
    return response.text

In [10]:
# Load website.html into a string
# with open('website.html', 'r') as file:
#     website_html = file.read()
    
# use selenium to open the browser and take a screenshot
browser = open_browser()

browser.get("https://www.dominos.com/en/")

html = browser.page_source

# take a screenshot of the website
screenshot = browser.save_screenshot("website.png")
img = PIL.Image.open('website.png')
url = browser.current_url
prompt = "I would like to order a pizza for delivery."

user_prompt = f"""
current_url: {url}
current_page: {html}
current_screenshot is attached
Output selectors for relevant elements (divs, inputs, and images) that are relevant to the user's request.x

user: {prompt}
"""
    
# Generate content using the prompt and the website HTML
response = generate_content_with_cycling_keys(user_prompt, system_prompt_interpret, img)
if "```json" in response:
    response = response.split("```json")[1].split("```")[0]
obj = json.loads(response)
obj

[{'type': 'xpath', 'selector': "//a[@data-quid='mix-and-match-deal-button']"},
 {'type': 'src',
  'selector': '/static/1.88.2/images/tiles/mixAndMatchDeal/hero.webp'}]

In [ ]:
dom_elements = ""
for element in obj:
    if element['type'] == 'xpath':
        dom_elements += extract_elements_by_xpath(html, element["selector"])
        dom_elements += "\n"
    else:
        dom_elements += f"src: {element['selector']}\n"

In [ ]:
print(dom_elements)

['<a data-dpz-track-evt-name="SYO_Delivery" data-quid="start-your-order-delivery-cta" href="/en/restaurants?type=Delivery" class="css-14js2j3">Delivery</a>', '//a[@data-quid="start-your-order-delivery-cta"]']
['<a data-dpz-track-evt-name="SYO_Carryout" data-quid="start-your-order-carryout-cta" href="/en/restaurants?type=Carryout" class="css-14js2j3">Carryout</a>', '//a[@data-quid="start-your-order-carryout-cta"]']



In [ ]:
design = PIL.Image.open("design.png")
response = generate_content_with_cycling_keys(
    design_schema
    + "\n\n"
    + dom_elements
    + "\n\n"
    + '"Only output div, button, input, img, and select elements. Do not use Tailwind Classes\nBase Url for images (if any): https://dominos.com \n\n',
    system_prompt_generate,
)

In [ ]:
print(response)

```html
<div>
    <div type='button' class='bg-primary text-white font-bold py-2 px-4 rounded-full' special-id='//a[@data-quid="start-your-order-delivery-cta"]'>
        Delivery
    </div>
    <div type='button' class='bg-primary text-white font-bold py-2 px-4 rounded-full' special-id='//a[@data-quid="start-your-order-carryout-cta"]'>
        Carryout
    </div>
</div>
``` 

